# Проверка графа на двудольность

**Задача.**

Составить программы, проверяющие граф на двудольность с помощью

1. Поиска в ширину (задача А)
2. Поиска в глубину (задача Б)

**Набор данных задачи:**
 
Файл описывает ребра графа. Вершины обозначаются целыми положительными числами от 1 до 875714. Каждая строка обозначает одно ребро графа. Например, одиннадцатая строка («2 13019») указывает на то, что существует ребро, соединяющее вершину 2 и вершину 13019.

**Требования:**

1. Сравнить временные затраты для обеих задач.
2. Привести и обосновать асимптотическую оценку временной сложности.

## Предобработка и считывание входных данных

Для работы с форматом входных данных в виде списка рёбер можно преобразовать его в граф в виде словаря смежности.

In [3]:
from collections import defaultdict

def adjacency_list(edges):

    adjacency_list = defaultdict(set)
    
    for u, v in edges:
        adjacency_list[u].add(v)
        adjacency_list[v].add(u)  # Так как граф неориентированный
    return {node: list(neighbors) for node, neighbors in adjacency_list.items()}

In [4]:
def file(filename):
    edges = []
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split()  # Разделяем строку по пробелу
            if len(parts) == 2:  # Проверяем, что ровно два элемента
                edges.append((int(parts[0]), int(parts[1])))
    return edges

test = adjacency_list(file('test.txt'))
data = adjacency_list(file('data.txt'))
my = adjacency_list(file('my.txt'))

print(my)

{1: [10, 7], 7: [1, 3], 2: [8, 11], 8: [2, 5], 3: [10, 7], 4: [9, 12], 9: [4, 6], 5: [8, 11], 6: [9, 12], 10: [1, 3], 11: [2, 5], 12: [4, 6]}


## Проверка графа на двудольность поиском в ширину.

1. Граф представлен в виде словаря смежности: каждая вершина — ключ, а его значение — список соседей.
2. Массив color используется для хранения цвета каждой вершины (0 или 1). Если вершина ещё не посещена, она не имеет цвета.
3. Алгоритм BFS:
    - Начинаем с произвольной вершины, окрашиваем её в цвет 0 и помещаем в очередь.
    - При посещении каждой вершины окрашиваем её соседей в противоположный цвет.
    - Если обнаруживается сосед, который уже окрашен в тот же цвет, граф не двудольный.

In [5]:
from collections import deque

def is_bipartite_bfs(graph):
    
    color = {}
    
    for node in graph:

        if node not in color:

            queue = deque([node])
            color[node] = 0
            
            while queue:
                current = queue.popleft()
                
                for neighbor in graph[current]:

                    if neighbor not in color:
                        color[neighbor] = 1 - color[current]
                        queue.append(neighbor)

                    elif color[neighbor] == color[current]:
                        print(neighbor, current)
                        return False
    return True

**Результат и время выполнения алгоритма проверки графа на двудольность (BFS).**

На тестовом наборе данных:

In [6]:
import time

start_time = time.time()
bipartite = is_bipartite_bfs(test)
end_time = time.time()

print(f'Время выполнения алгоритма проверки графа на двудольность (BFS): {end_time - start_time} секунд')
print(f'Двудольный ли граф? - {bipartite}')

5 4
Время выполнения алгоритма проверки графа на двудольность (BFS): 0.0 секунд
Двудольный ли граф? - False


На усложненном наборе данных:

In [7]:
import time

start_time = time.time()
bipartite = is_bipartite_bfs(data)
end_time = time.time()

print(f'Время выполнения алгоритма проверки графа на двудольность (BFS): {end_time - start_time} секунд')
print(f'Двудольный ли граф? - {bipartite}')

1 1
Время выполнения алгоритма проверки графа на двудольность (BFS): 0.0 секунд
Двудольный ли граф? - False


Мой набор данных (граф является двудольным):

In [8]:
import time

start_time = time.time()
bipartite = is_bipartite_bfs(my)
end_time = time.time()

print(f'Время выполнения алгоритма проверки графа на двудольность (BFS): {end_time - start_time} секунд')
print(f'Двудольный ли граф? - {bipartite}')

Время выполнения алгоритма проверки графа на двудольность (BFS): 0.0 секунд
Двудольный ли граф? - True


## Проверка графа на двудольность методом в глубину.

1. Каждый узел окрашивается в текущий цвет (0 или 1).
2. Если сосед ещё не окрашен, рекурсивно вызываем dfs для него, меняя цвет на противоположный.
3. Если сосед уже окрашен и его цвет совпадает с текущим узлом, граф не двудольный.
4. Проверка всех узлов:
    - Учитываются несвязные компоненты графа. Если узел ещё не окрашен, начинаем новый вызов dfs.


In [9]:
def is_bipartite_dfs(graph):

    color = {}

    def dfs(node, current_color):

        color[node] = current_color
        
        for neighbor in graph[node]:
            if neighbor not in color:
                # Рекурсивно вызываем DFS для соседей, с противоположным цветом
                if not dfs(neighbor, 1 - current_color):
                    return False
            elif color[neighbor] == color[node]:
                print(neighbor, node)
                # Найдено ребро между вершинами одного цвета
                return False
        return True

    for node in graph:  # Проходим по всем вершинам (учитываем несвязные графы)
        if node not in color:
            if not dfs(node, 0):
                return False

    return True

**Результат и время выполнения алгоритма проверки графа на двудольность (DFS).**

На тестовом наборе данных:

In [10]:
import time

start_time = time.time()
bipartite = is_bipartite_dfs(test)
end_time = time.time()

print(f'Время выполнения алгоритма проверки графа на двудольность (DFS): {end_time - start_time} секунд')
print(f'Двудольный ли граф? - {bipartite}')

2 5
Время выполнения алгоритма проверки графа на двудольность (DFS): 0.0 секунд
Двудольный ли граф? - False


На усложненном наборе данных:

In [11]:
import time

start_time = time.time()
bipartite = is_bipartite_dfs(data)
end_time = time.time()

print(f'Время выполнения алгоритма проверки графа на двудольность (DFS): {end_time - start_time} секунд')
print(f'Двудольный ли граф? - {bipartite}')

1 1
Время выполнения алгоритма проверки графа на двудольность (DFS): 0.0 секунд
Двудольный ли граф? - False


Мой набор данных (граф является двудольным):

In [14]:
import time

start_time = time.time()
bipartite = is_bipartite_dfs(my)
end_time = time.time()

print(f'Время выполнения алгоритма проверки графа на двудольность (BFS): {end_time - start_time} секунд')
print(f'Двудольный ли граф? - {bipartite}')

Время выполнения алгоритма проверки графа на двудольность (BFS): 0.0 секунд
Двудольный ли граф? - True
